# [NTDS'18] milestone 3: spectral graph theory
[ntds'18]: https://github.com/mdeff/ntds_2018

[Michaël Defferrard](http://deff.ch), [EPFL LTS2](https://lts2.epfl.ch)

## Students

* Team: `10`
* Students: `Cionca Alexandre, De Goumoëns Frédéric, Donzier Paul, Fluhr Hugo`
* Dataset: `TMDb dataset (subset of IMDb)`

## Rules

* Milestones have to be completed by teams. No collaboration between teams is allowed.
* Textual answers shall be short. Typically one to two sentences.
* Code has to be clean.
* You cannot import any other library than we imported.
* When submitting, the notebook is executed and the results are stored. I.e., if you open the notebook again it should show numerical results and plots. We won't be able to execute your notebooks.
* The notebook is re-executed from a blank state before submission. That is to be sure it is reproducible. You can click "Kernel" then "Restart & Run All" in Jupyter.

## Objective

The goal of this milestone is to get familiar with the graph Laplacian and its spectral decomposition.

## 0 Load your network

In [ ]:
%matplotlib inline

If you get a `No module named 'sklearn'` error when running the below cell, install [scikit-learn](https://scikit-learn.org) with `conda install scikit-learn` (after activating the `ntds_2018` environment).

In [ ]:
import numpy as np
from scipy import sparse
import scipy.sparse.linalg
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

import pandas as pd
import json

#for testing, to be removed
from mpl_toolkits.mplot3d import Axes3D

import networkx as nx

import pygsp as mdeff


In [ ]:
features = pd.read_csv('../Milestone_1/fits.csv')

# Importing our dataset that will be updated as adjacency is updated so that we can use it to answer Question 13 (is our clustering representing
# some feature of the movies?)
json_columns = ['genres','crew','production_companies','production_countries']
for column in json_columns:
     features[column] = features[column].apply(json.loads)
        
        
features['genres_name']=""

for i in range(len(features.title)):
    strs=[]
    for ii in range(len(features.genres[i])):
        strs.append(features.genres[i][ii].get('name'))
    features.at[i,'genres_name']=strs

#list(features) used to determine which columns to drop
features = features.drop(columns='Unnamed: 0')
len(features)

Let's denote your graph as $\mathcal{G} = (\mathcal{V}, \mathcal{E}, A)$, where $\mathcal{V}$ is the set of nodes, $\mathcal{E}$ is the set of edges, $A \in \mathbb{R}^{N \times N}$ is the (weighted) adjacency matrix, and $N = |\mathcal{V}|$ is the number of nodes.

Import the adjacency matrix $A$ that you constructed in the first milestone.
(You're allowed to update it between milestones if you want to.)

In [ ]:
adjacency = np.load("../Milestone_1/adj_act.npy")

n_nodes =   len(adjacency)
print(n_nodes,'nodes')
#adjacency[adjacency<0.05]=0

#adjacency=np.where(adjacency!=0,1.,0.)

In [ ]:
# We see from the adjacency matrix that there are some movies that don't have any connections to any other movies,
# these nodes don't present any interest to us so we remove them from both the dataframe and the adjacency matrix
deg=np.sum(adjacency,0)
deg_bin=np.argwhere(deg==0).flatten()

adjacency_red = np.delete(adjacency,deg_bin,0)
adjacency_red = np.delete(adjacency_red,deg_bin,1)

fits_red=features.drop(deg_bin)
fits_red=fits_red.reset_index(drop=True)

# We need to update n_nodes since we have dropped some movies
n_nodes=len(adjacency_red)
n_nodes
adjacency=adjacency_red
len(adjacency)

## 1 Graph Laplacian

### Question 1

From the (weighted) adjacency matrix $A$, compute both the combinatorial (also called unnormalized) and the normalized graph Laplacian matrices.

Note: if your graph is weighted, use the weighted adjacency matrix. If not, use the binary adjacency matrix.

For efficient storage and computation, store these sparse matrices in a [compressed sparse row (CSR) format](https://en.wikipedia.org/wiki/Sparse_matrix#Compressed_sparse_row_.28CSR.2C_CRS_or_Yale_format.29).

In [ ]:
# We first create the degree vector of our graph :
degrees= np.sum(adjacency,0)

D_sqrt_inv= np.diag(1/np.sqrt(degrees))

laplacian_combinatorial = sparse.csr_matrix(np.diag(degrees)-adjacency)
laplacian_normalized = sparse.csr_matrix(D_sqrt_inv @ laplacian_combinatorial @ D_sqrt_inv)
plt.figure(figsize=(10,10))
plt.subplot(121)
plt.spy(laplacian_combinatorial,markersize=0.1)
plt.subplot(122)
plt.spy(laplacian_normalized,markersize=0.1)
lap_un=laplacian_combinatorial.toarray()

Use one of them as the graph Laplacian $L$ for the rest of the milestone.
We however encourage you to run the code with both to get a sense of the difference!

In [ ]:
#laplacian =  laplacian_combinatorial
laplacian = laplacian_normalized

# We chose the normalized laplacian because its useful to get the number of clusters (see question 10)

### Question 2

Compute the eigendecomposition of the Laplacian $L = U^\top \Lambda U$, where the columns $u_k \in \mathbb{R}^N$ of $U = [u_1, \dots, u_N] \in \mathbb{R}^{N \times N}$ are the eigenvectors and the diagonal elements $\lambda_k = \Lambda_{kk}$ are the corresponding eigenvalues.

Make sure that the eigenvalues are ordered, i.e., $0 = \lambda_1 \leq \lambda_2 \leq \dots \leq \lambda_N$.

In [ ]:
# Scipy linalg (not scipy.sparse.linagl!!!)
vals_norm,vects_norm = scipy.linalg.eigh(laplacian.toarray())
print(vals_norm[0])

# Numpy linalg
eigenvalues, eigenvectors = np.linalg.eigh(laplacian.toarray())
print(eigenvalues[0])

vals_un,vects_un = np.linalg.eigh(lap_un)

#assert eigenvectors.shape == (n_nodes, n_nodes)

In [ ]:
%timeit vals_norm,vects_norm = scipy.linalg.eigh(laplacian.toarray())

In [ ]:
%timeit eigenvalues, eigenvectors = np.linalg.eigh(laplacian.toarray())

In [ ]:
#print(eigenvectors[:,0])
#print(vects[:,0])
sum(vects_norm[:,0]**2) #gives the norm of the vector which is equal to 1

Justify your choice of eigensolver.

**Your answer here.**

https://stackoverflow.com/questions/6684238/whats-the-fastest-way-to-find-eigenvalues-vectors-in-python

### Question 3

We can write $L = S S^\top$. What is the matrix $S$? What does $S^\top x$, with $x \in \mathbb{R}^N$, compute?

S is the incidence matrix. It contains the information about the orientation of the edges, it gives a correspondace between
vertices and edges. S is of size N x M, N being number of vertices and M number of edges. S(i,j)= +1 / -1 if node i is 
the outgoing / ingoing node of edge j, S(i,j)= 0 othewise.
$S^\top x$ computes the gradient of x. It gives the derivative of x at each edge, $S^\top x$ is a vector of size M.

### Question 4

Show that $\lambda_k = \| S^\top u_k \|_2^2$, where $\| \cdot \|_2^2$ denotes the squared Euclidean norm (a.k.a. squared $L^2$ norm).

Remembering that $L = S S^\top $ and that $\| S^\top u_k \|_2^2 = u_k^\top S S^\top u_k  $. We use the definition of eigenvalues and eigenvectors :

$L u_k = \lambda_k u_k$

Left-multiplying both sides by $u_k^\top$ gives :

$ u_k^\top L u_k = u_k^\top \lambda_k u_k$

$\lambda_k $ being a scalar we can rewrite :

$ u_k^\top L u_k = \lambda_k u_k^\top  u_k$

Assuming that the eigenvectors are normalized (which is the case for most eigensolvers) we get :

$ u_k^\top L u_k = \lambda_k $

Which is equivalent to :

$ \lambda_k = u_k^\top L u_k = u_k^\top S S^\top u_k = \| S^\top u_k \|_2^2 $



What does the quantity $\| S^\top x \|_2^2$ tell us about $x$?

This quantity is the square of the norm of the gradient of x, meaning that it tells us how much the signal x varies over the graph.
It is an indicator of the smoothness of x over the graph.

### Question 5

What is the value of $u_0$, both for the combinatorial and normalized Laplacians?

In [ ]:
vects_norm[:,0] # for normalized, same as combinatorial but normalized by degree? in lecture : L*u = lambda*D*u 
vects_un[:,0]

**Your annswer here.**

Should be indicating vector of giant component, so full of 1's (normalized) for our network which is connected

### Question 6

Look at the spectrum of the Laplacian by plotting the eigenvalues.
Comment on what you observe.

In [ ]:
eig_min=0
eig_max=20
x = range(eig_min,eig_max,1)

plt.figure(figsize=(15,5))


plt.subplot(121)
#plt.plot(vals)
plt.scatter(x,vals_norm[eig_min:eig_max])
#plt.scatter(vals[:exe],list(np.ones((exe,1))))

plt.subplot(122)
#plt.plot(eigenvalues)
plt.scatter(x,eigenvalues[eig_min:eig_max])
#plt.scatter(eigenvalues[:exe],list(np.ones((exe,1))))

**Your answer here.**

How many connected components are there in your graph? Answer using the eigenvalues only.

In [ ]:
# Your code here.
# 1 since first eigenvalue is zero

Is there an upper bound on the eigenvalues, i.e., what is the largest possible eigenvalue? Answer for both the combinatorial and normalized Laplacians.

**Your answer here.**

## 3 Laplacian eigenmaps

*Laplacian eigenmaps* is a method to embed a graph $\mathcal{G}$ in a $d$-dimensional Euclidean space.
That is, it associates a vector $z_i \in \mathbb{R}^d$ to every node $v_i \in \mathcal{V}$.
The graph $\mathcal{G}$ is thus embedded as $Z \in \mathbb{R}^{N \times d}$.

In [ ]:
# We use our bfs to get the index of the nodes that are part of the giant component (attaignable from node 0)

def bfs(adjacency, source_node):
    # Explored is an array where explored[a_certain_node]= 0 when this a_certain_node cannot be reached by source node
    # or = 1 when this a_certain_node can be reached
    n_nodes=len(adjacency)
    explored=np.full(n_nodes,0)
    # Store nodes to explore in a list
    to_explore = [source_node]
    
    # The source node is already explored
    explored[source_node] = 1  
    
    # Stay in loop while there are still nodes to explore
    while to_explore:
       # Pop first node frome the queue and add it to to the explored nodes
        current_node = to_explore.pop(0)
        explored[current_node] = 1
        # Copy the line of the matrix corresponding to the current node to see its connections
        neighbours = np.argwhere(adjacency[current_node]).flatten()
        
        # Set the neighbours of current_node to be explored
        for i in range(len(neighbours)):
            if not explored[neighbours[i]]:
                to_explore.append(neighbours[i])
                explored[neighbours[i]]=1
    
    return explored

In [ ]:
vects_un[:,1]
idx=np.where(np.abs(vects_un[:,1]) <1e-3 , 1, 0)
bf=np.where(bfs(adjacency,0)==0)
# create an array containing the index of the nodes which are not in the giant component
idx_non_giant=np.argwhere(1-idx)
len(idx_non_giant)


In [ ]:
adjacency_red = np.delete(adjacency,idx_non_giant,0)
adjacency_red = np.delete(adjacency_red,idx_non_giant,1)
feat=fits_red
feat=feat.drop(idx_non_giant.flatten())
feat=feat.reset_index(drop=True)
len(adjacency_red)

In [ ]:
Graph = nx.from_numpy_array(adjacency)
Gr_2 = nx.from_numpy_array(adjacency_red)

giant=max(nx.connected_component_subgraphs(Graph), key=len)
print(nx.is_connected(giant))
print(nx.is_connected(Gr_2))


adjacency_gc=np.asarray(nx.to_numpy_matrix(giant))
print(np.max(adjacency_red-adjacency_gc))

#nx.write_gexf(Gr_2, 'movies_keywords.gexf')

##print(nx.connected_components(Graph))
len(adjacency_gc)

In [ ]:
degrees=np.sum(adjacency_red,0)
D_sqrt_inv= np.diag(1/np.sqrt(degrees))

laplacian_combinatorial = sparse.csr_matrix(np.diag(degrees)-adjacency_red)
laplacian_normalized = sparse.csr_matrix(D_sqrt_inv @ laplacian_combinatorial @ D_sqrt_inv)
plt.figure(figsize=(10,10))
plt.subplot(121)
plt.spy(laplacian_combinatorial,markersize=0.1)
plt.subplot(122)
plt.spy(laplacian_normalized,markersize=0.1)
laplacian=laplacian_normalized

### Question 7

What do we use Laplacian eigenmaps for? (Or more generally, graph embeddings.)

**Your answer here.**

Embed in an Euclidian Space, for visualisation for example, the visualisation has to "show" properties of the graph
Laplacian Eigenmaps are also useful for dimensionality reduction using neighborhood information of the dataset (each data point is a node of the graph)

### Question 8

Embed your graph in $d=2$ dimensions with Laplacian eigenmaps.
Try with and without re-normalizing the eigenvectors by the degrees, then keep the one your prefer.

**Recompute** the eigenvectors you need with a partial eigendecomposition method for sparse matrices.
When $k \ll N$ eigenvectors are needed, partial eigendecompositions are much more efficient than complete eigendecompositions.
A partial eigendecomposition scales as $\Omega(k |\mathcal{E}|$), while a complete eigendecomposition costs $\mathcal{O}(N^3)$ operations.

In [ ]:
# We first create the degree vector of our graph :
degrees=np.sum(adjacency_gc,0)
D_sqrt_inv= np.diag(1/np.sqrt(degrees))

laplacian_combinatorial = sparse.csr_matrix(np.diag(degrees)-adjacency_gc)
laplacian_normalized = sparse.csr_matrix(D_sqrt_inv @ laplacian_combinatorial @ D_sqrt_inv)

n_eig=13

# Eigenvalues and eigenvectors are computed using the 
vals_s, eigenvectors_s = sparse.linalg.eigsh(laplacian_normalized,n_eig,maxiter=400000, which='SM')

# Eigenvectors matrix is Re-normalized by the degrees
eigenV = D_sqrt_inv @ eigenvectors_s

In [ ]:
n_min=0
n_max=30
ran=range(len(vals_s))
plt.scatter(ran[n_min:n_max],vals_s[n_min:n_max])

In [ ]:
k=1

plt.figure(figsize=(20,80))

for i in range(1, len(eigenvectors_s[0])-1,1):
    for j in range(i+1,len(eigenvectors_s[0])-1,1):
        #x=eigenvectors_s[:,i]
        #y=eigenvectors_s[:,j]
        x=eigenV[:,i]
        y=eigenV[:,j]
        plt.subplot(20,5,k)
        strange='i:'+ str(i) +'j:'+ str(j)
        plt.title(strange)
        plt.scatter(x, y, alpha=0.25, c=K_means.labels_, cmap='hsv')
        k+=1
        


Plot the nodes embedded in 2D. Comment on what you see.

In [ ]:
plt.figure(figsize=(12,12))

x1= eigenvectors_s[:,1]
y1= eigenvectors_s[:,9]
x2= eigenvectors_s[:,4]
y2= eigenvectors_s[:,10]

plt.subplot(2,2,1)
plt.scatter(x1, y1, alpha=0.25, c=K_means.labels_, cmap='hsv');
plt.title("No re-normalization (1,9)")
plt.xlabel("2nd Eigenvector")
plt.ylabel("9th Eigenvector")

plt.subplot(2,2,2)
plt.scatter(x2, y2, alpha=0.25, c=K_means.labels_, cmap='hsv');
plt.title("No re-normalization (4,10)")
plt.xlabel("11th Eigenvector")
plt.ylabel("9th Eigenvector")

x1= eigenV[:,1]
y1= eigenV[:,9]
x2= eigenV[:,4]
y2= eigenV[:,10]

plt.subplot(2,2,3)
plt.scatter(x1, y1, alpha=0.25, c=K_means.labels_, cmap='hsv');
plt.title("Re-normalization (1,9)")
plt.xlabel("2nd Eigenvector")
plt.ylabel("9th Eigenvector")

plt.subplot(2,2,4)
plt.scatter(x2, y2, alpha=0.25, c=K_means.labels_, cmap='hsv');
plt.title("Re-normalization (4,10)")
plt.xlabel("11th Eigenvector")
plt.ylabel("9th Eigenvector")

**Your answer here.**

### Question 9

What does the embedding $Z \in \mathbb{R}^{N \times d}$ preserve?

**Your answer here.**

LES DISTANCES

A COMPLETER <br>
Points close to each other on the manifold are mapped close to each other in the low-dimensional space, preserving local distances.
https://en.wikipedia.org/wiki/Nonlinear_dimensionality_reduction#Laplacian_eigenmaps

## 2 Spectral clustering

*Spectral clustering* is a method to partition a graph into distinct clusters.
The method associates a feature vector $z_i \in \mathbb{R}^d$ to every node $v_i \in \mathcal{V}$, then runs [$k$-means](https://en.wikipedia.org/wiki/K-means_clustering) in the embedding space $\mathbb{R}^d$ to assign each node $v_i \in \mathcal{V}$ to a cluster $c_j \in \mathcal{C}$, where $k = |\mathcal{C}|$ is the number of desired clusters.

### Question 10

Choose $k$ and $d$. How did you get to those numbers?

We choose k from observing the spectrum of our graph. As we saw in the lecture, the index of the eigenvalue (of the normalized Laplacian) at which there is a gap (a discontinuity in the spectrum) is a good choice for the number of clusters.

### Question 11

1. Embed your graph in $\mathbb{R}^d$ as $Z \in \mathbb{R}^{N \times d}$.
   Try with and without re-normalizing the eigenvectors by the degrees, then keep the one your prefer.
1. If you want $k=2$ clusters, partition with the Fiedler vector. For $k > 2$ clusters, run $k$-means on $Z$. Don't implement $k$-means, use the `KMeans` class imported from scikit-learn.

In [ ]:
n_eigV=9
k=10

#H=eigenvectors_s[:,1:n_eigV]
H=eigenV[:,1:n_eigV]

print(np.shape(H))

K_means=KMeans(n_clusters = k, random_state=0).fit(H)

labs=[]

for i in range(k):
    labs.append(len(np.argwhere(K_means.labels_ == i)))
    print(i,':',labs[i])

labs=sorted(labs, reverse=True)
labels=K_means.labels_
## turn a graph into a set of vectors and run k-means on it,
## rows are samples, columns are features

### Question 12

Use the computed cluster assignment to reorder the adjacency matrix $A$.
What do you expect? What do you observe?

In [ ]:
# Your code here.
## order by clusters, the order is no longer arbitrary, comment on wether this shows a structure in the network
## maybe the clusters will give sequels? MCU, Pirates of the carribeans...
## the clusters number order doesn't matter
sort_by_cluster=np.argsort(labels)
feat['cluster'] = labels
feat['index_by_cluster']=np.argsort(sort_by_cluster)
feat.sort_values('index_by_cluster')

adjacency_sorted=adjacency_red[sort_by_cluster,:]
adjacency_sorted=adjacency_sorted[:,sort_by_cluster]

plt.figure(figsize=(10,10))
plt.spy(adjacency_sorted, markersize=0.1)
plt.title('adjacency matrix')

In [ ]:
feat.sort_values('cluster',ascending=False)

**Your answer here.**

### Question 13

If you have ground truth clusters for your dataset, compare the cluster assignment from spectral clustering to the ground truth.
A simple quantitative measure is to compute the percentage of nodes that have been correctly categorized.
If you don't have a ground truth, qualitatively assess the quality of the clustering.

Ground truth clusters are the "real clusters".
For example, the genre of musical tracks in FMA, the category of Wikipedia articles, the spammer status of individuals, etc.
Look for the `labels` in the [dataset descriptions](https://github.com/mdeff/ntds_2018/tree/master/projects/README.md).

In [ ]:
# Your code here.

### Question 14

Plot the cluster assignment (one color per cluster) on the 2D embedding you computed above with Laplacian eigenmaps.

In [ ]:
# Your code here.

### Question 15

Why did we use the eigenvectors of the graph Laplacian as features? Could we use other features for clustering?

**Your answer here.**